In [1]:
from Bio import SeqIO, pairwise2
import numpy as np
import time
import pandas as pd

/Users/mohanavenkataphaneendrareddyalla/opt/anaconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
# Define the scoring system
match = 10
mismatch = -3
gap_penalty = -5


In [3]:

#Filter genomes without 'N' and ensure uniqueness
def filter_genomes(input_file):
    genomes = list(SeqIO.parse(input_file, "fasta"))
    unique_genomes = set()  # to ensure uniqueness
    filtered_genomes = []

    for genome in genomes:
        seq_str = str(genome.seq)
        if 'N' not in seq_str and seq_str not in unique_genomes:
            unique_genomes.add(seq_str)
            filtered_genomes.append(genome)
    
    return filtered_genomes[:20]  # Return the first 20 unique genomes

# Load and filter genomes without 'N'
genome_file = '/Users/mohanavenkataphaneendrareddyalla/Selected_Unique_COVID19_Genomes_Asia_new1.fasta'
genomes = filter_genomes(genome_file)
print(f"Filtered {len(genomes)} genomes without 'N'.")
print(genomes)



Filtered 20 genomes without 'N'.
[SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='NC_045512.2', name='NC_045512.2', description='NC_045512.2', dbxrefs=[]), SeqRecord(seq=Seq('GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATG...GAG'), id='PQ283576.1', name='PQ283576.1', description='PQ283576.1', dbxrefs=[]), SeqRecord(seq=Seq('GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATG...GAG'), id='PQ283577.1', name='PQ283577.1', description='PQ283577.1', dbxrefs=[]), SeqRecord(seq=Seq('GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATG...GAG'), id='PQ267978.1', name='PQ267978.1', description='PQ267978.1', dbxrefs=[]), SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...AAA'), id='BS016240.1', name='BS016240.1', description='BS016240.1', dbxrefs=[]), SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...TCC'), id='BS016241.1', name='BS016241.1', description='BS016241.1', dbxrefs=[]), SeqRecord(seq=S

In [4]:
# Function for pairwise alignment using the Needleman-Wunsch algorithm
def pairwise_alignment(seq1, seq2):
    alignments = pairwise2.align.globalms(seq1, seq2, match, mismatch, gap_penalty, gap_penalty)
    return alignments[0][2]  # Return the alignment score (score is at index 2)


In [5]:
# Function to extract the first 'length' base pairs from each genome
def extract_subsequence(genomes, length):
    return [str(genome.seq[:length]) for genome in genomes]


In [6]:
# Function to generate the pairwise alignment score matrix for a given sequence length
def generate_alignment_matrix(genomes, length):
    subsequences = extract_subsequence(genomes, length)
    n = len(subsequences)
    matrix = np.zeros((n, n))  # Initialize an n x n matrix to store alignment scores

    start_time = time.time()  # Start timing
    for i in range(n):
        for j in range(i, n):
            score = pairwise_alignment(subsequences[i], subsequences[j])
            matrix[i][j] = score  # Fill the score matrix
            matrix[j][i] = score  # Symmetric matrix
    end_time = time.time()  # End timing

    run_time = end_time - start_time  # Calculate the running time
    return matrix, run_time


In [7]:
# Define the lengths for alignment: 500bp, 1000bp, 1500bp, 2000bp
lengths = [500, 1000, 1500, 2000]
matrices = {}  # Dictionary to store alignment matrices
times = {}     # Dictionary to store running times

# Perform pairwise alignment for each segment size
for length in lengths:
    alignment_matrix, run_time = generate_alignment_matrix(genomes, length)
    matrices[length] = alignment_matrix
    times[length] = run_time
    
    print(f"\nAlignment matrix for {length}bp:\n", alignment_matrix)
    df = pd.DataFrame(alignment_matrix)
    df.to_csv(f"alignment_matrix_{length}bp.csv", index=False, header=False)


Alignment matrix for 500bp:
 [[5000. 3887. 3874. 3887. 3907. 3907. 3907. 3847. 3887. 4567. 3907. 4507.
  4467. 4854. 3907. 4741. 4507. 4487. 4287. 3987.]
 [3887. 5000. 4987. 5000. 4980. 4980. 4980. 4960. 5000. 4320. 4980. 4380.
  4420. 4020. 4980. 4120. 4380. 4400. 4600. 4900.]
 [3874. 4987. 5000. 4987. 4967. 4967. 4967. 4947. 4987. 4307. 4967. 4367.
  4407. 4007. 4967. 4107. 4367. 4387. 4587. 4887.]
 [3887. 5000. 4987. 5000. 4980. 4980. 4980. 4960. 5000. 4320. 4980. 4380.
  4420. 4020. 4980. 4120. 4380. 4400. 4600. 4900.]
 [3907. 4980. 4967. 4980. 5000. 5000. 5000. 4940. 4980. 4340. 5000. 4400.
  4440. 4040. 5000. 4140. 4400. 4420. 4620. 4920.]
 [3907. 4980. 4967. 4980. 5000. 5000. 5000. 4940. 4980. 4340. 5000. 4400.
  4440. 4040. 5000. 4140. 4400. 4420. 4620. 4920.]
 [3907. 4980. 4967. 4980. 5000. 5000. 5000. 4940. 4980. 4340. 5000. 4400.
  4440. 4040. 5000. 4140. 4400. 4420. 4620. 4920.]
 [3847. 4960. 4947. 4960. 4940. 4940. 4940. 5000. 4960. 4280. 4940. 4340.
  4380. 3980. 4940. 4

In [8]:
# Print running times for each segment length
for length, run_time in times.items():
    print(f"Running time for {length}bp segment: {run_time:.2f} seconds")


Running time for 500bp segment: 1.72 seconds
Running time for 1000bp segment: 6.17 seconds
Running time for 1500bp segment: 14.32 seconds
Running time for 2000bp segment: 25.21 seconds


In [9]:
# Basic Statistical Analysis
def compute_statistics(matrix):
    # Extract the upper triangular part of the matrix to avoid duplicates
    flat_scores = matrix[np.triu_indices(len(matrix), k=1)]
    mean_score = np.mean(flat_scores)
    min_score = np.min(flat_scores)
    max_score = np.max(flat_scores)
    std_dev = np.std(flat_scores)
    
    return mean_score, min_score, max_score, std_dev

# Perform statistics for each length
for length in matrices:
    mean_score, min_score, max_score, std_dev = compute_statistics(matrices[length])
    print(f"\nStatistics for {length}bp segments:")
    print(f"Mean alignment score: {mean_score:.2f}")
    print(f"Minimum alignment score: {min_score:.2f}")
    print(f"Maximum alignment score: {max_score:.2f}")
    print(f"Standard deviation: {std_dev:.2f}")



Statistics for 500bp segments:
Mean alignment score: 4572.19
Minimum alignment score: 3847.00
Maximum alignment score: 5000.00
Standard deviation: 347.69

Statistics for 1000bp segments:
Mean alignment score: 9555.51
Minimum alignment score: 8821.00
Maximum alignment score: 10000.00
Standard deviation: 350.92

Statistics for 1500bp segments:
Mean alignment score: 14552.91
Minimum alignment score: 13821.00
Maximum alignment score: 15000.00
Standard deviation: 351.30

Statistics for 2000bp segments:
Mean alignment score: 19547.71
Minimum alignment score: 18821.00
Maximum alignment score: 20000.00
Standard deviation: 350.10
